##### データセットの場所やバッチサイズなどの定数値の設定

In [3]:
import os
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'


# 使用するデバイス
# Paperspace Gradient などで GPU を利用する場合は DEVICE = 'cuda:0' とする
DEVICE = 'cpu'

# 全ての訓練データを一回ずつ使用することを「1エポック」として，何エポック分学習するか
# どの程度の値が良いかは，色々試してみないと分からない
N_EPOCHS = 50

# 学習時のバッチサイズ
# 基本的に大きい方が高速，しかし GPU 利用時にメモリ不足（out of memory）でエラーになる可能性は高まる
BATCH_SIZE = 100

# 訓練データセット（画像ファイルリスト）のファイル名
TRAIN_DATASET_CSV = './Weather/weather_train.csv'

# テストデータセット（画像ファイルリスト）のファイル名
TEST_DATASET_CSV = './Weather/weather_test.csv'

# 学習結果の保存先フォルダ
MODEL_DIR = './MLP_models'

# 学習結果のニューラルネットワークの保存先
MODEL_FILE = os.path.join(MODEL_DIR, 'weather_predictor_model.pth')

##### ニューラルネットワークモデルの定義

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# 天気認識AIを実現するニューラルネットワーク
# 多層パーセプトロン（MLP）のサンプル
class WeatherPredictor(nn.Module):

    def __init__(self):
        super(WeatherPredictor, self).__init__()

        # 全結合層1: 入力（2次元）→ 1層目（10パーセプトロン）
        self.fc1 = nn.Linear(in_features=2, out_features=10)

        # 全結合層2: 1層目（10パーセプトロン）→ 2層目（10パーセプトロン）
        self.fc2 = nn.Linear(in_features=10, out_features=10)

        # 全結合層3: 2層目（10パーセプトロン）→ 出力（3次元）
        self.fc3 = nn.Linear(in_features=10, out_features=3)

    def forward(self, x):
        h = self.fc1(x) # 全結合層1にデータを入力（線形和を計算）
        h = torch.tanh(h) # 活性化関数tanh
        h = self.fc2(h) # 続いて全結合層2に通す（線形和を計算）
        h = F.relu(h) # 活性化関数 ReLU
        y = self.fc3(h) # 最後に全結合層3に通す
        return y

##### 訓練データセットの読み込み

In [5]:
import pickle
from torch.utils.data import DataLoader, random_split
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, 訓練データセットを用意
dataset = CSVBasedDataset(
    filename = TRAIN_DATASET_CSV,
    items = [
        ['平均気温', '平均湿度'], # X
        '天気概況' # Y
    ],
    dtypes = [
        'float', # Xの型
        'label' # Yの型
    ]
)
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'wb') as fdicts_file:
    pickle.dump(dataset.forward_dicts, fdicts_file)

# 認識対象のクラス数を取得
n_classes = len(dataset.forward_dicts[1])

# 訓練データセットを分割し，一方を検証用に回す
dataset_size = len(dataset)
valid_size = int(0.05 * dataset_size) # 全体の 5% を検証用に
train_size = dataset_size - valid_size # 残りの 95% を学習用に
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# 訓練データおよび検証用データをミニバッチに分けて使用するための「データローダ」を用意
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習処理の実行

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import tqdm
from mylib.visualizers import LossVisualizer
from mylib.visualizers import ClassifierVisualizer


# 学習経過を可視化するか否か（可視化する場合は True，しない場合は False にする）
VISUALIZE = True

# 学習経過を可視化する準備
if VISUALIZE:
    perm = np.random.permutation(train_size)[:200] # 訓練データから200サンプルをランダムに抽出して可視化
    visualizer = ClassifierVisualizer(n_classes=3, clabels=['sunny', 'cloudy', 'rainy'], hrange=[-5, 40], vrange=[20, 110], hlabel='temperature (deg C)', vlabel='humidity (%)', bins=10)
    samples_for_visualization = (np.asarray([train_dataset[i][0].numpy() for i in perm]), np.asarray([train_dataset[i][1] for i in perm]))

# ニューラルネットワークの作成
model = WeatherPredictor().to(DEVICE)

# 最適化アルゴリズムの指定（ここでは SGD でなく Adam を使用）
optimizer = optim.Adam(model.parameters())

# 損失関数：クロスエントロピー損失を使用
loss_func =  nn.CrossEntropyLoss()

# 損失関数値を記録する準備
loss_viz = LossVisualizer(['train loss', 'valid loss'])

# 勾配降下法による繰り返し学習
for epoch in range(N_EPOCHS):

    print('Epoch {0}:'.format(epoch + 1))

    # 学習
    model.train()
    sum_loss = 0
    for X, Y in tqdm(train_dataloader): # X, Y は CSVBasedDataset クラスの __getitem__ 関数の戻り値に対応
        for param in model.parameters():
            param.grad = None
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X) # 入力値 X を現在のニューラルネットワークに入力し，出力の推定値を得る
        loss = loss_func(Y_pred, Y) # 損失関数の現在値を計算
        loss.backward() # 誤差逆伝播法により，個々のパラメータに関する損失関数の勾配（偏微分）を計算
        optimizer.step() # 勾配に沿ってパラメータの値を更新
        sum_loss += float(loss) * len(X)
    avg_loss = sum_loss / train_size
    loss_viz.add_value('train loss', avg_loss) # 訓練データに対する損失関数の値を記録
    print('train loss = {0:.6f}'.format(avg_loss))

    # 検証
    model.eval()
    sum_loss = 0
    n_failed = 0
    with torch.inference_mode():
        for X, Y in tqdm(valid_dataloader):
            X = X.to(DEVICE)
            Y = Y.to(DEVICE)
            Y_pred = model(X)
            loss = loss_func(Y_pred, Y)
            sum_loss += float(loss) * len(X)
            n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    avg_loss = sum_loss / valid_size
    loss_viz.add_value('valid loss', avg_loss) # 検証用データに対する損失関数の値を記録
    accuracy = (valid_size - n_failed) / valid_size
    print('valid loss = {0:.6f}'.format(avg_loss))
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')

    # 学習経過の可視化
    if VISUALIZE:
        visualizer.show(model, class_colors=[[255, 0, 0], [127, 127, 0], [0, 0, 255]], samples=samples_for_visualization, title='Epoch {0}'.format(epoch + 1))

# 学習結果のニューラルネットワークモデルをファイルに保存
model = model.to('cpu')
torch.save(model.state_dict(), MODEL_FILE)

# 損失関数の記録をファイルに保存
loss_viz.save(v_file=os.path.join(MODEL_DIR, 'loss_graph.png'), h_file=os.path.join(MODEL_DIR, 'loss_history.csv'))

##### テストデータセットの読み込み

In [4]:
import pickle
from torch.utils.data import DataLoader
from mylib.data_io import CSVBasedDataset


# CSVファイルを読み込み, テストデータセットを用意
with open(os.path.join(MODEL_DIR, 'fdicts.pkl'), 'rb') as fdicts_file:
    fdicts = pickle.load(fdicts_file)
test_dataset = CSVBasedDataset(
    filename = TEST_DATASET_CSV,
    items = [
        ['平均気温', '平均湿度'], # X
        '天気概況' # Y
    ],
    dtypes = [
        'float', # Xの型
        'label' # Yの型
    ],
    fdicts = fdicts
)
test_size = len(test_dataset)

# 認識対象のクラス数を取得
n_classes = len(test_dataset.forward_dicts[1])

# テストデータをミニバッチに分けて使用するための「データローダ」を用意
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

##### 学習済みニューラルネットワークモデルのロード

In [7]:
import torch


# ニューラルネットワークモデルとその学習済みパラメータをファイルからロード
model = WeatherPredictor()
model.load_state_dict(torch.load(MODEL_FILE))

##### 単一データに対するテスト処理の実行

In [ ]:
import torch


model = model.to(DEVICE)
model.eval()

# index 番目のテストデータをニューラルネットワークに入力してみる
class_labels = test_dataset.reverse_dicts[1]
while True:
    print('index?: ', end='')
    val = input()
    if val == 'exit': # 'exit' とタイプされたら終了
        break
    index = int(val)
    x, y = test_dataset[index]
    x = x.reshape(1, *x.size()).to(DEVICE)
    with torch.inference_mode():
        y_pred = model(x)
    y_pred = torch.argmax(y_pred, dim=1)
    print('')
    print('平均気温 == {0:.1f}(度), 平均湿度 == {1:.1f}(%)'.format(x[0, 0], x[0, 1]))
    print('estimated:', class_labels[int(y_pred)])
    print('ground truth:', class_labels[int(y)])
    print('')

##### 全てのデータに対するテスト処理の実行

In [ ]:
import torch
from tqdm import tqdm


model.eval()

# テストデータセットを用いて認識精度を評価
n_failed = 0
with torch.inference_mode():
    for X, Y in tqdm(test_dataloader):
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)
        Y_pred = model(X)
        n_failed += torch.count_nonzero(torch.argmax(Y_pred, dim=1) - Y) # 推定値と正解値が一致していないデータの個数を数える
    accuracy = (test_size - n_failed) / test_size
    print('accuracy = {0:.2f}%'.format(100 * accuracy))
    print('')